In [1]:
 !pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to c:\users\sarthak\appdata\local\temp\pip-req-build-is6vwval
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=7683 sha256=44616d4b476a289ad100a9b77a2d8d181b3f821623687a5ded9e2660ae23df5d
  Stored in directory: C:\Users\SARTHAK\AppData\Local\Temp\pip-ephem-wheel-cache-gp89y58d\wheels\fd\71\47\e04f208ad659a295ccb69022e14de7d20ccc6fc30a5a935f09
Successfully built preprocess-kgptalkie


In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [4]:
import preprocess_kgptalkie as ps

In [5]:
df = pd.read_csv('imdb_reviews.txt', sep = '\t', header = None)

In [6]:
df.columns = ['reviews', 'sentiment']

In [8]:
df.head()

,reviews,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [7]:
x = "A very, very, very slow-moving, aimless movie"
ps.remove_special_chars(x)
ps.remove_accented_chars(x)
ps.cont_exp(x)
ps.remove_emails(x)
ps.remove_html_tags(x)
ps.remove_urls(x)
ps.make_base(x)

ps.spelling_correction(x)

TextBlob("A very, very, very slow-moving, aimless movie")

In [8]:
df['reviews'] = df['reviews'].apply(lambda x: ps.cont_exp(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_special_chars(x))

df['reviews'] = df['reviews'].apply(lambda x: ps.remove_accented_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_emails(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_html_tags(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_urls(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.make_base(x))


In [9]:
df['reviews'] = df['reviews'].apply(lambda x: str(x).lower())


In [13]:
df

,reviews,sentiment
0,a very very very slowmove aimless movie about ...,0
1,not sure who was more lose the flat character ...,0
2,attempt artiness with black white and clever c...,0
3,very little music or anything to speak of,0
4,the good scene in the movie was when gerardo i...,1
...,...,...
743,i just get bored watch jessice lange take her ...,0
744,unfortunately any virtue in this film producti...,0
745,in a word it is embarrassing,0
746,exceptionally bad,0


In [10]:
X = df['reviews']
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

In [16]:
X_train.shape, X_test.shape

((598,), (150,))

In [12]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver = 'liblinear'))
])

In [13]:
hyperparameters = {
    'tfidf__max_df': (0.5, 1.0),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'char', 'char_wb'),
    'clf__penalty': ('l2', 'l1'),
    'clf__C': (1,2)
}

In [14]:
clf = GridSearchCV(pipe, hyperparameters, n_jobs=-1, cv = None)

In [15]:
%%time
clf.fit(X_train, y_train)

Wall time: 15.7 s


GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'clf__C': (1, 2), 'clf__penalty': ('l2', 'l1'),
                         'tfidf__analyzer': ('word', 'char', 'char_wb'),
                         'tfidf__max_df': (0.5, 1.0),
                         'tfidf__ngram_range': ((1, 1), (1, 2)),
                         'tfidf__use_idf': (True, False)})

In [16]:
clf.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(C=2, solver='liblinear'))])

In [17]:
clf.best_params_

{'clf__C': 2,
 'clf__penalty': 'l2',
 'tfidf__analyzer': 'word',
 'tfidf__max_df': 1.0,
 'tfidf__ngram_range': (1, 1),
 'tfidf__use_idf': True}

In [16]:
clf.best_score_

0.7708263305322129

# SVM Model


In [17]:
from sklearn.svm import LinearSVC

In [18]:
pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())
])

In [19]:
hyperparameters = {
    'tfidf__max_df': (0.5, 1.0),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'char', 'char_wb'),
    'clf__C': (1,2,2.5,3)
}

In [20]:
clf = GridSearchCV(pipe, hyperparameters, n_jobs=-1, cv = 5)

In [21]:
%%time
clf.fit(X_train, y_train)

Wall time: 8.93 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', LinearSVC())]),
             n_jobs=-1,
             param_grid={'clf__C': (1, 2, 2.5, 3),
                         'tfidf__analyzer': ('word', 'char', 'char_wb'),
                         'tfidf__max_df': (0.5, 1.0),
                         'tfidf__ngram_range': ((1, 1), (1, 2)),
                         'tfidf__use_idf': (True, False)})

In [24]:
clf.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer(max_df=0.5)),
                ('clf', LinearSVC(C=1))])

In [25]:
clf.best_params_

{'clf__C': 1,
 'tfidf__analyzer': 'word',
 'tfidf__max_df': 0.5,
 'tfidf__ngram_range': (1, 1),
 'tfidf__use_idf': True}

In [22]:
clf.best_score_

0.782563025210084

# Model Testing

In [23]:
x = ['this is great moview. i loved it', 'i have watched this movie. plot is straight. return my money']

In [24]:
clf.predict(x)

array([1, 0], dtype=int64)

In [25]:
import pickle as pkl

In [26]:
pkl.dump(clf, open('model.pkl', 'wb'))

In [27]:
clf = pkl.load(open('model.pkl','rb'))

In [28]:
clf.predict(x)

array([1, 0], dtype=int64)